Loading the data.

In [128]:
import pandas as pd
ori_data_path = r'D:\美赛\比赛ing\Code\Problem_C_Data_Wordle.xlsx'
ori_data = pd.read_excel(ori_data_path)
Column_name = ori_data.iloc[0,:].values
ori_data.columns = Column_name
ori_data = ori_data.drop(axis=0,index=0)
ori_data = ori_data.iloc[:,1:]
ori_data.head()

,Date,Contest number,Word,Number of reported results,Number in hard mode,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X)
1,2022-12-31 00:00:00,560,manly,20380,1899,0,2,17,37,29,12,2
2,2022-12-30 00:00:00,559,molar,21204,1973,0,4,21,38,26,9,1
3,2022-12-29 00:00:00,558,havoc,20001,1919,0,2,16,38,30,12,2
4,2022-12-28 00:00:00,557,impel,20160,1937,0,3,21,40,25,9,1
5,2022-12-27 00:00:00,556,condo,20879,2012,0,2,17,35,29,14,3


In [129]:
attr_path = r'D:\美赛\比赛ing\Code\data_attr_no_pos.csv'
attr = pd.read_csv(attr_path)

attr = attr.drop('Unnamed: 0',axis=1)
attr = attr.drop('Contest number',axis=1)

count_mean = attr.iloc[:,1].mean()
count_std = attr.iloc[:,1].std()
attr['count'] = (attr['count'] - count_mean)/count_std

en_mean = attr.iloc[:,9].mean()
en_std = attr.iloc[:,9].std()
attr['entropy'] = (attr['entropy'] - en_mean)/en_std

attr.head()

,Word,count,noun,verb,adj,adv,pron,num,prep,entropy,...,s,t,u,v,w,x,y,z,vowel_num,total_Word_class
0,manly,-0.287384,0,0,1,1,0,0,0,0.386198,...,0.0,0.0,0.0,0.000,0.0,0.0,1.97,0.0,1,2
1,molar,-0.294364,1,0,1,0,0,0,0,0.698246,...,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,2,2
2,havoc,-0.282446,1,1,0,0,0,0,0,0.247338,...,0.0,0.0,0.0,0.978,0.0,0.0,0.00,0.0,2,2
3,impel,-0.300813,0,1,0,0,0,0,0,1.649583,...,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,2,1
4,condo,-0.228084,1,0,0,0,0,0,0,-0.383127,...,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,2,1


The word_to_vector function:

In [130]:
word_dict = {}
for i in range(359):
    word = attr.iloc[i,0]
    word = word.strip()
    word_dict[word] = []
    for letter in list(word):
        feature = attr.loc[i,letter]
        # print(i,letter,word)
        word_dict[word].append(feature)
print(word_dict['grate'])


[2.02, 5.99, 8.17, 9.06, 12.7]


The label data:

In [131]:
target = ori_data['Number in hard mode']/ori_data.iloc[:,3]
target = pd.DataFrame(target) 
target.columns = ['Percentage']
target = target.iloc[::-1]
target.reset_index(drop=True,inplace=True)
target = target * 100

Prepare the training data as [ wordvector, total_Word_class, vowel_num, count, entropy, percentage ] with 10 input and 7 outputs

In [132]:
train_data_1 = {}
for i in range(359):
    if sum(ori_data.iloc[i,-7:]) > 105 or sum(ori_data.iloc[i,-7:]) < 95:
        continue
    word = attr.iloc[i,0].strip()
    if len(list(word)) != 5:
        continue
    # print(word)
    train_data_1[word] = word_dict[word]
    
    train_data_1[word].append(attr.loc[i,'total_Word_class']) 
    train_data_1[word].append(attr.loc[i,'vowel_num']) 
    train_data_1[word].append(attr.loc[i,'count']) 
    train_data_1[word].append(attr.loc[i,'entropy']) 

    train_data_1[word].append(target.iloc[i,:].values[0]) 

    train_data_1[word].append(ori_data.iloc[i,-7])
    # print(attr.iloc[i,-7])
    train_data_1[word].append(ori_data.iloc[i,-6]) 
    # print(attr.iloc[i,-6])
    train_data_1[word].append(ori_data.iloc[i,-5]) 
    train_data_1[word].append(ori_data.iloc[i,-4]) 
    train_data_1[word].append(ori_data.iloc[i,-3]) 
    train_data_1[word].append(ori_data.iloc[i,-2]) 
    train_data_1[word].append(ori_data.iloc[i,-1]) 

print(train_data_1['grate'])
print(len(train_data_1))

[2.02, 5.99, 8.17, 9.06, 12.7, 2, 2, -0.2941134695897084, 0.6826346903801769, 6.555838481125617, 1, 14, 29, 28, 16, 8, 3]
356


In [134]:
valid_key = []
for key in train_data_1.keys():
    valid_key.append(key)
print(len(valid_key))

356


After data processing, we build the deep learing model:

In [135]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
# import warnings
# warnings.filterwarnings('ignore')

In [194]:
# About hyperparameters
seq_len = 10            # 每个样本包含的序列长度
tar_len = 7             # 准备通过样本进行多个预测
n_features = seq_len    # 输入的特征数，实际上为10
n_hidden = 32           # hidden_size,也就是d_model
n_layers = 2            # 层数
n_heads = 4             # 注意力头数
dropout = 0.5           # 随即失活数量
n_epochs = 200          # 训练轮数
position_len = 1000     # 最长序列长度

# define the dataset about time series
class DistributionDataset(Dataset):
    def __init__(self, data, seq_len,tar_len):
        self.data = data
        self.seq_len = seq_len
        self.tar_len = tar_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if len(self.data) == 72:
            word = valid_key[idx + 284]
        # print(word)
        else:
            word = valid_key[idx]
        return self.data[word][:self.seq_len], self.data[word][self.seq_len:]

# 网上调整函数
def collate_fn(batch):
    # batch是一个列表，其中是一个一个的元组，每个元组是dataset中_getitem__的结果
    batch = list(zip(*batch))
    train_data = batch[0]
    label = batch[1]
    # print(label)
    # label = label.astype(np.int32)
    # time_sery = time_sery.astype(np.int32)
    train_data = torch.tensor(train_data,dtype=torch.float32)
    label = torch.tensor(label,dtype=torch.float32)
    # print(label.shape)
    del batch
    return  train_data, label

# class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=position_len):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

class TransformerModel(nn.Module):
    def __init__(self, n_features, n_hidden, n_layers, n_heads, dropout):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(n_features, n_hidden)
        # self.position_encoding = PositionalEncoding(n_hidden)
        self.encoder_layer = nn.TransformerEncoderLayer(n_hidden, n_heads, 2048, dropout=dropout)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, n_layers)
        self.decoder_layer = nn.TransformerDecoderLayer(n_hidden, n_heads,dim_feedforward=2048, dropout=dropout)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, n_layers)
        self.fc = nn.Linear(n_hidden, tar_len)

    def forward(self, x):
        x = x.to(torch.float32)
        # print(x.shape)
        x = x.unsqueeze(1) 
        # print(x.shape)
        # x = torch.transpose(x)
        # print(x.shape)
        x = self.embedding(x)
        # print(x.shape)
        # x = self.position_encoding(x)
        # print(x.shape)
        x = self.encoder(x)
        # print(x.shape)
        x = self.decoder(x, x)
        # print(x.shape)
        x = self.fc(x)
        # print(x.shape)
        x = x.squeeze(1)        # [batch_size, target_len]
        # print(x.shape)
        return x

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # print(len(inputs))
        inputs = torch.stack([torch.Tensor(i).to(device) for i in inputs])
        targets = torch.stack([torch.Tensor(i).to(torch.float32).to(device) for i in targets])
        # print(targets.shape)
        optimizer.zero_grad()
        outputs = model(inputs).to(torch.float32)
        # print(outputs.shape)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    return train_loss

def evaluate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = torch.stack([torch.Tensor(i).to(device) for i in inputs])
            targets = torch.stack([torch.Tensor(i).to(device) for i in targets])
            outputs = model(inputs).float()
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    return val_loss

Training:

In [154]:
# Formorlise the data
data = train_data_1
train_size = int(len(data) * 0.8)
train_data = {}
for idx, item in enumerate(train_data_1):
    if idx + 1 == train_size:
        train_data[item] = []
        train_data[item] += (train_data_1[item])
        break
    else:
        train_data[item] = []
        train_data[item] += (train_data_1[item])

# print(len(train_data))
val_data = {}
for idx, item in enumerate(train_data_1):
    if idx + 1 <= train_size:
        continue
    else:
        val_data[item] = []
        val_data[item] += (train_data_1[item])
# print(len(val_data))

In [195]:
# create the dataset and loader
train_dataset = DistributionDataset(train_data, seq_len,tar_len)
val_dataset = DistributionDataset(val_data, seq_len,tar_len)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16,collate_fn=collate_fn)

# Initialise the model and optimizer
model = TransformerModel(n_features=n_features, n_hidden=n_hidden, n_layers=n_layers, n_heads=n_heads, dropout=dropout)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# 训练模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
train_loss_list = []
val_loss_list = []
val_loss_compare = 0
count = 0
for epoch in range(n_epochs):
    
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    # if val_loss <= 20 and epoch > 100: 
        # break
    val_loss_compare = val_loss
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# plot train_loss and val_loss
plt.plot(val_loss_list,label='val')
plt.plot(train_loss_list,label='train')
plt.legend()
plt.show()

Epoch 1/500, Train Loss: 332.3934, Val Loss: 293.5478
Epoch 11/500, Train Loss: 142.0074, Val Loss: 111.3002
Epoch 21/500, Train Loss: 43.4047, Val Loss: 28.5123
Epoch 31/500, Train Loss: 30.2454, Val Loss: 21.3998
Epoch 41/500, Train Loss: 28.9113, Val Loss: 21.3801
Epoch 51/500, Train Loss: 27.9206, Val Loss: 21.0622
Epoch 61/500, Train Loss: 27.5036, Val Loss: 21.9375
Epoch 71/500, Train Loss: 25.2580, Val Loss: 23.5311
Epoch 81/500, Train Loss: 26.0653, Val Loss: 22.7093
Epoch 91/500, Train Loss: 24.2599, Val Loss: 22.8858
Epoch 101/500, Train Loss: 24.2713, Val Loss: 22.3048
Epoch 111/500, Train Loss: 23.0956, Val Loss: 23.2071
Epoch 121/500, Train Loss: 23.7813, Val Loss: 21.4477
Epoch 131/500, Train Loss: 23.5555, Val Loss: 25.1860
Epoch 141/500, Train Loss: 22.5946, Val Loss: 22.1910
Epoch 151/500, Train Loss: 22.4300, Val Loss: 22.0930
Epoch 161/500, Train Loss: 23.6995, Val Loss: 22.9584
Epoch 171/500, Train Loss: 21.3826, Val Loss: 25.0371
Epoch 181/500, Train Loss: 22.0359,

KeyboardInterrupt: 

In [196]:
min(val_loss_list)

20.541527938842773

Prediction:

In [197]:
def prediction(model, pre_data, device):
    model.eval()
    with torch.no_grad():
        test_input = torch.Tensor(pre_data).unsqueeze(0).to(device)
        # print(len(pre_data))
        test_output = model(test_input).cpu().numpy()[0]
        # print(test_output.cpu().numpy())
        return test_output

In [201]:
threshold = 0.3
perfect = 0
good = 0
fail = 0
for idx in range(len(val_data)):
    score = 0
    word = valid_key[idx + 284]
    pred = prediction(model,train_data_1[word][:10],device)
    truth = train_data_1[word][10:]
    for j in range(7):
        if abs(pred[j] - truth[j])/(truth[j]+0.0000001) < threshold:
            score += 1
    if score >= 3:
        good += 1
        if score >= 5:
            perfect += 1
    else:
        fail += 1

print(perfect, good, fail)

29 55 17


In [200]:
print('pred:', prediction(model,train_data_1['cloth'][:10],device).tolist())
print('truth:', train_data_1['cloth'][10:])

pred: [0.7947500348091125, 8.904172897338867, 28.913963317871094, 35.367122650146484, 21.61078453063965, 9.220895767211914, 2.0617473125457764]
truth: [1, 8, 33, 34, 17, 7, 1]
